### 1. 原始数据读取
- 并不是把所有图像全部读进内存！
- 而是把所有图像的`路径`和`类别`归纳和梳理出来！
- img_path
- img_label

In [1]:
"""
    尝试读取 train 
"""
import os
train_root = os.path.join("gesture", "train")
train_paths = []
train_labels = []

for label in os.listdir(train_root):
    label_root = os.path.join(train_root, label)
    for file in os.listdir(label_root):
        file_path = os.path.join(label_root, file)
        train_paths.append(file_path)
        train_labels.append(label)

NotADirectoryError: [WinError 267] 目录名称无效。: 'gesture\\train\\._eight'

In [2]:
"""
    尝试读取 test 
"""
import os
test_root = os.path.join("gesture", "test")
test_paths = []
test_labels = []

for label in os.listdir(test_root):
    label_root = os.path.join(test_root, label)
    for file in os.listdir(label_root):
        file_path = os.path.join(label_root, file)
        test_paths.append(file_path)
        test_labels.append(label)

In [3]:
# 构建 标签字典 label dict
labels = ["zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]
label2idx = {label: idx for idx, label in enumerate(labels)}
idx2label = {idx: label for label, idx in label2idx.items()}

### 2. 批量化打包
- 继承 `Dataset`，自定义一个数据集
- 实例化 `DataLoader`

In [4]:
# 引入必要的工具类
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image
from torchvision import transforms
import torch

In [5]:
class GestureDataset(Dataset):
    """
        自定义手势识别数据集
    """
    def __init__(self, X, y):
        """
            初始化
        """
        self.X = X
        self.y = y

    def __getitem__(self, idx):
        """
            实现：
                - 按下标来索引一个样本
        """
        # 获取图像路径
        img_path = self.X[idx]
        # 读取图像
        img = Image.open(fp=img_path)
        # 统一大小
        img = img.resize((32, 32))
        # 转张量 [C, H, W]
        # [0, 1]
        img = transforms.ToTensor()(img)
        # [-1, 1]
        img = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])(img)

        # 读取标签
        img_label = self.y[idx]
        # 标签转 id
        img_idx = label2idx.get(img_label)
        # 转张量
        label = torch.tensor(data=img_idx, dtype=torch.long)

        return img, label

    
    def __len__(self):
        """
            返回该数据集的样本个数
        """
        return len(self.X)

In [6]:
# 训练集加载器
train_dataset = GestureDataset(X=train_paths, y=train_labels)
train_dataloader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=16)
# 测试集加载器
test_dataset = GestureDataset(X=test_paths, y=test_labels)
test_dataloader = DataLoader(dataset=test_dataset, shuffle=False, batch_size=32)

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [7]:
# 测试
for X, y in test_dataloader:
    print(X.shape)
    print(y.shape)
    break

torch.Size([32, 3, 32, 32])
torch.Size([32])


### 3. 搭建模型

In [7]:
import torch
from torch import nn

In [9]:
class Model(nn.Module):
    """
        自定义一个神经网络
    """
    def __init__(self, in_channels=3, n_classes=10):
        """
            初始化
        """
        super(Model, self).__init__()

        # 1. 特征抽取
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, 
                               out_channels=6, 
                               kernel_size=5,
                               stride=1,
                               padding=0),
            nn.MaxPool2d(kernel_size=2, 
                                stride=2,
                                padding=0),
            nn.Conv2d(in_channels=6,
                              out_channels=16,
                              kernel_size=5,
                              stride=1,
                              padding=0),
            nn.MaxPool2d(kernel_size=2, 
                               stride=2,
                               padding=0)
        )

        # 2. 分类输出
        self.classifier = nn.Sequential(
            nn.Flatten(start_dim=1, end_dim=-1),
            nn.Linear(in_features=400, out_features=120),
            nn.Linear(in_features=120, out_features=84),
            nn.Linear(in_features=84, out_features=n_classes)
        )

    def forward(self, x):
        """
            前向传播
        """
        # 1. 先做特征抽取
        x = self.feature_extractor(x)
        # 2. 再做分类回归
        x = self.classifier(x)
        return x

### 4. 训练过程

In [10]:
# 设置训练轮次
epochs = 50
# 设备
device = "cuda" if torch.cuda.is_available() else "cpu"
# 实例化模型
model = Model()
model.to(device=device)
# 优化器
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)
# 损失函数
loss_fn = nn.CrossEntropyLoss()

In [11]:
def train():
    for epoch in range(epochs):
        for batch_X, batch_y in train_dataloader:
            # 1. 数据搬家
            batch_X = batch_X.to(device=device)
            batch_y = batch_y.to(device=device)
            # 2. 正向传播
            y_pred = model(batch_X)
            # 3. 计算损失
            loss = loss_fn(y_pred, batch_y)
            # 4. 反向传播
            loss.backward()
            # 5. 优化一步
            optimizer.step()
            # 6. 清空梯度
            optimizer.zero_grad()
        # 7. 打印损失
        print(loss.item())           

In [12]:
train()

0.3870279788970947
0.3729983866214752
0.5501367449760437
0.5294032692909241
0.07594510167837143
0.07750631868839264
0.32474711537361145
0.047036413103342056
0.25252389907836914
0.04849974066019058
0.03929836302995682
0.00987283606082201
0.018879974260926247
0.003361350391060114
0.0035993927158415318
0.013037455268204212
0.022112084552645683
0.017116395756602287
0.01115716714411974
0.0062713115476071835


KeyboardInterrupt: 

In [ ]:
"""
    1, 过程监控
    2，早停设置（在测试集上，如果连续N=3轮没有性能提升，则停止训练）
    3，模型的best.pt和last.pt
    4，加载预训练模型 last.pt
    5，模型加载、推理流程
    6,...
    
"""